## A wrapper for OPA within AQUA

In [20]:
from aqua import OPAgenerator

Initialize the OPA generator class, feeding it with the most of the information you need. Which variable, which model source and where to store the data.

In [21]:
opaopa = OPAgenerator(model="IFS", exp="tco2559-ng5", source="ICMGG_atm2d", var='2t', frequency='daily',
            outdir='/home/b/b382289/scratch/one_pass', tmpdir='/home/b/b382289/scratch/one_pass', definitive=False)


2023-05-12 13:31:43 :: opa_generator :: WARNING  -> IMPORTANT: no file will be created, this is a dry run
2023-05-12 13:31:43 :: opa_generator :: WARNING  -> Variable(s) to be processed: 2t
2023-05-12 13:31:43 :: create_folder :: WARNING  -> Folder /home/b/b382289/scratch/one_pass/IFS/tco2559-ng5/daily already exists
2023-05-12 13:31:43 :: create_folder :: WARNING  -> Folder /home/b/b382289/scratch/one_pass already exists


Then you can retrieve the data using the `Reader`. This is based on a streaming generator so far since we are using data from NextGEMS, will be later replaced with direct access from `FDB`. It also gave a look at the data so that it can infer the timedelta

In [22]:
opaopa.retrieve()

This will launch the OPA computation, if the `defitinive` flag is `true` it will take ages and probably break the notebook. Otherwise print only the sequential access to the data from the `Reader`.
Note that to print the sequential access the loglevel should be set to `INFO` or `DEBUG`.

In [23]:
opaopa.generate_opa()

2023-05-12 13:31:53 :: opa_generator :: WARNING  -> Setting up OPA at daily frequency for variable 2t...
2023-05-12 13:31:53 :: opa_generator :: WARNING  -> Initializing the OPA
2023-05-12 13:31:53 :: opa_generator :: WARNING  -> Initializing the streaming generator...


{'stat': 'mean', 'stat_freq': 'daily', 'output_freq': 'monthly', 'time_step': 60, 'variable': '2t', 'save': True, 'checkpoint': True, 'checkpoint_filepath': '/home/b/b382289/scratch/one_pass', 'out_filepath': '/home/b/b382289/scratch/one_pass/IFS/tco2559-ng5/daily', 'checkpoint_file': '/home/b/b382289/scratch/one_pass/checkpoint_2t_daily_monthly_mean.pkl'}


You can then add an entry to the catalog so that you can access these data from somewhere else.

In [24]:
opaopa.create_catalog_entry()

2023-05-12 13:33:35 :: opa_generator :: WARNING  -> Creating catalog entry IFS tco2559-ng5 tmp-opa-daily


In [25]:
from aqua import Reader
reader = Reader(model="IFS", exp="tco2559-ng5", source="tmp-opa-daily",areas=False)

In order to really access and work with data a run with `definitive` set to `True` is needed.

This remove the entries (and in the future also the files), once all the computation have been safely concluded

In [26]:
opaopa.clean()

2023-05-12 13:33:35 :: opa_generator :: WARNING  -> Removing catalog entry IFS tco2559-ng5 tmp-opa-daily
